In [3]:
import requests
import numpy as np
import pandas as pd
import json
import re
import emoji
import nltk
import flair
nltk.download('words')
words = set(nltk.corpus.words.words())



class tweeter():
    def __init__(self,ticker,BEARER_TOKEN,max_result):
        self.ticker = ticker
        self.BEARER_TOKEN = BEARER_TOKEN
        self.max_results = str(max_result)
        
    def clean_tweet(self,tweet):
        tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
        tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
        tweet = " ".join(tweet.split())
        tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
        tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
        tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
                         if w.lower() in words or not w.isalpha())
        return tweet
    
    def get_data(self,tweet):
        data = {'id':tweet['id'],
                'created_at':tweet['created_at'],
                'text':tweet['text']}
        return data
    
    def create_data_frame(self):
        
        endpoint = 'https://api.twitter.com/2/tweets/search/recent'
        headers = {'authorization': f"Bearer {self.BEARER_TOKEN}"}
        params = {
                    'query': '(self.ticker) (lang:en)',
                    'max_results': self.max_results,
                    'tweet.fields': 'created_at,lang'
                    }
        response = requests.get(endpoint,
                            params=params,
                            headers=headers)
        response.json()
        df = pd.DataFrame()
        for tweet in response.json()['data']:
            row = self.get_data(tweet)
            df.append(row , ignore_index = True)
            
        df['text'] = df['text'].map(lambda x : self.clean_tweet(x))
        return df
    
    def adding_prob_and_sentiment(self,df):
        probs = []
        sentiments = []
        sentiment_model = flair.models.TextClassifier.load('en-sentiment')
        for tweet in df['text'].to_list():
            sentence = flair.data.Sentence(tweet)
            sentiment_model.predict(sentence)
            probs.append(sentence.label[0].score)
            sentiments.append(sentence.label[0].value)
        df['probs'] = probs
        df['sentiments'] = sentiments
        
        return df
        
BEARERTOKEN = 'AAAAAAAAAAAAAAAAAAAAAD5IUwEAAAAAYRmAUWOXGmKinw5EJY4c0mK4L%2Fk%3DfQ4d0x6AHSjU0HYYbya8lnLEE5QRsIyWakBJmiTVqscmtYEYHi'    
tweeter = tweeter('tsla',BEARERTOKEN,100)
df = tweeter.create_data_frame()

[nltk_data] Downloading package words to /Users/roji/nltk_data...
[nltk_data]   Package words is already up-to-date!


KeyError: 'data'